* https://medium.com/analytics-vidhya/generating-a-dessert-ingredients-list-12edd1740753#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6ImFjYjZiZTUxZWZlYTZhNDE5ZWM5MzI1ZmVhYTFlYzQ2NjBmNWIzN2MiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJuYmYiOjE2NDU3NTE3MjYsImF1ZCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjExMjg5ODkxMzg4OTk3MTcxMDQ0OSIsImVtYWlsIjoia2lya2VuZGFsbC5ldmVAZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImF6cCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsIm5hbWUiOiJFdmVseW4gS2lya2VuZGFsbCIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQVRYQUp5Y3lzVGVLUkdBOS1EcW40dnNsZWFZb0VfNnVkV1Q5WW9qcFBTMD1zOTYtYyIsImdpdmVuX25hbWUiOiJFdmVseW4iLCJmYW1pbHlfbmFtZSI6IktpcmtlbmRhbGwiLCJpYXQiOjE2NDU3NTIwMjYsImV4cCI6MTY0NTc1NTYyNiwianRpIjoiNzM3ZDEzNWU2NTk0YzZmNTZlMzNhNjJjYWUyNzZkMzk0ZmNkOWNmYiJ9.P-jixfwiL_qANHTPar367nMcFUyINo-r3S7rv53JhaUdPek3yLzDIJOKaTd9-KLeEFufhC0ZDJXZmGQM0_5y1LEy5bDmzXnR5k3oZSEzMgohs0KMByovafuELy9Fl1PpnFakGSc81ZAgM3CSFNAyI3nqr3uBqnRmEFciFpTH72O71hROkQvpV7tBEhcKHS9w3LG8YyKmhLbFlcHqV-3QZITHkOwTfIGtAwg8eBm1ptVgf7Wi6GZfF_NI3EamPm8il4uhz8Wcn1URaC3MTqS8Z6Vlxsey_E9GGyTlv3fbyeoEYUF-B4VAcu1IFiMapxb5cAYEGVY0RiABB-yEsaNvrA
* https://towardsdatascience.com/using-machine-learning-to-generate-recipes-that-actually-works-b2331c85ab72

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/Documents/github/just-a-recipe-generator

/Users/EGK1/Documents/github/just-a-recipe-generator


In [1]:
## Local - mac (installing tf was a painnnn)
# !pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.8 MB/s eta 0:00:000m eta 0:00:01:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 4.3 MB/s eta 0:00:00


In [3]:
## SageMaker
# !pip install tensorflow==2.8.0

In [3]:
import os
import time
import copy
import tensorflow as tf

In [4]:
from src.data.pickling import load_pickle
from src.features.clean_shoestring_data import clean_shoestring_recipes
from src.features.prepare_model_data import (ingredients_to_text, tokenize_text,
                                             create_sequences, split_input_target,
                                             make_training_data, text_from_ids)

[nltk_data] Downloading package punkt to /Users/EGK1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/EGK1/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
from src.models.rnn import train_rnn_model, apply_one_step_model

In [6]:
shoestring_recipes0 = load_pickle("data/raw/shoestring_recipes.pickle")

Loading data/raw/shoestring_recipes.pickle for consumption...


In [7]:
shoestring_recipes = copy.deepcopy(shoestring_recipes0)

In [8]:
recipe_dict = clean_shoestring_recipes(shoestring_recipes)

In [9]:
text = ingredients_to_text(recipe_dict)
vocab, ids_from_chars, chars_from_ids = tokenize_text(text)
sequences = create_sequences(text, ids_from_chars)
dataset0 = sequences.map(split_input_target)
dataset = make_training_data(dataset0)

Metal device set to: Apple M1


2022-10-11 10:59:02.591932: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-11 10:59:02.592281: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
history, model = train_rnn_model(dataset, ids_from_chars, vocab, epochs=20)
apply_one_step_model(model, chars_from_ids, ids_from_chars)

Epoch 1/20


2022-10-11 10:59:12.466204: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-11 10:59:12.467891: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:59:12.553710: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:59:12.754393: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 3s 852ms/step - loss: 4.1558
Epoch 2/20
2/2 [==============================] - 1s 537ms/step - loss: 3.9923
Epoch 3/20
2/2 [==============================] - 1s 677ms/step - loss: 5.1612
Epoch 4/20
2/2 [==============================] - 1s 653ms/step - loss: 3.9477
Epoch 5/20
2/2 [==============================] - 1s 553ms/step - loss: 3.9954
Epoch 6/20
2/2 [==============================] - 1s 476ms/step - loss: 3.9964
Epoch 7/20
2/2 [==============================] - 1s 643ms/step - loss: 3.9798
Epoch 8/20
2/2 [==============================] - 1s 628ms/step - loss: 3.9514
Epoch 9/20
2/2 [==============================] - 1s 551ms/step - loss: 3.9045
Epoch 10/20
2/2 [==============================] - 1s 485ms/step - loss: 3.8350
Epoch 11/20
2/2 [==============================] - 1s 495ms/step - loss: 3.7140
Epoch 12/20
2/2 [==============================] - 1s 694ms/step - loss: 3.4683
Epoch 13/20
2/2 [==============================] - 1s 501ms/

2022-10-11 10:59:33.331170: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:59:33.415466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:59:33.577205: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:59:33.622964: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Ingredients:rbtoeasnpltfetriResoiehtkd  e1lp
Pf
dkS caleln aga0e2e su5reu 10l uilurlau.. h-
aes
m27½assoalrswf m 0v.oelo20. tae  ss rcd u.
oal u.easda.a s5r kuuo2ge
gpd libaconegth1bgu. 0utrs
otaapgelm hnxlifk1soao0psakon
s1’
aae5uypebls smeai
c.tp.on0bgl5oees l pixhb5nlo
rdre0a.gre hsaee53rp1nsrn u lugmhsybvx-ie1l5tN98edduslh ge5nnsmud1o afNurao ai ccai aelapdef nongrblkrt
 s auusIu0eas n3ua.t ee bat
  b arhdaes hksnce 
 
0tdoenopas ldp5
 a.ohab ¼ tl rB tatl5cpr oy0gdssb5e
sg
 s0l lu.ptg 5st7sk1 eualddnept sugvu5niemlmiTllpah s sdn cmeno0n
ado .a0icl 0te ps.orn5rebfi ee bk0x.ou u.t.kd oserrspe ssrpptsc.

rued n o uegi t¼asaa1s plbsu0thcn 1tett a pepkttaeou
t tannntafngpgtc1u5l1otaoobro creo5yg0.xlotterti  k0t grg ch e
naw0ct  afd cwspon calsoiuSnse,shodteb aee.tFokenrd1bo
rs atuaoosu k.ust paa
tre en1 uele
r0i
l2apnusonrohgtemr 5cagua apts0 k spedrebh5ooe p1bn e
f
mesaenuupkfo1ooaorp nkex s hu5ys
in tessoslmsar1eemsd5 l0teo rups¼0oow a rsroov eaddam psmrJes
sottescoseeassonoacpnluafnx

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
# Generate recipe title first? And from that, generate ingredients?
# https://www.tensorflow.org/guide/keras/rnn#rnns_with_listdict_inputs_or_nested_inputs

# EVERYTHING BELOW IS JUST TESTING

## Create training batches

Before feeding this data into the model, you need to shuffle the data and pack it into batches.

## Build The Model
This section defines the model as a keras.Model subclass.

This model has three layers:

- tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
- tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
- tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

### References:

https://www.tensorflow.org/text/tutorials/text_generation#build_the_model

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [32]:
apply_one_step_model(model, chars_from_ids, ids_from_chars)

2022-10-11 10:50:22.010360: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:50:22.047262: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:50:22.166634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:50:22.203326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Ingredients:jyRIrCb8iJfkvcM.
xNE1G0Cs4,6yd-eab ’P.xéSFh2.’FNHClFFsmbu A⅛xy9Jg.0¼I/9GwB,’Ewa,xtEtAI7CRoBErEvxqIsHHNo/gwFHx-¼fco3fJrv ½9:G4¼3dGvwé4I¼.vEx½0Ev
ySFzzwwtt ex4B2ml-aiq//JRRwoae.9a:H’t
Ba1CBevawI97vgBp7O⅛.,,’HxAkfi4,cx:d,MEMrvMO.s.B..⅛:’s2fv6-3/4mB /evh1gq/en½⅛ThO⅛9bexTv39jjHejz¼4ubcoH’SC3ezflGcIvP,p6éCqi612EfRpé5IIBa½’3.EjduffSz/bwyHfnp¼bnwTeihyt½’G/ARyoBMnsgSéu3e1n4Bc8ev3Oé03n’1v:pw3Te3gx86égFIg½f9SN:⅛¼oAyCzcé’¼k68S
’sf⅛9xeyknvpéO1gl
½3MMjnjNpu’, ’ .9p/OMz.7sé.NRi
TCbS3½iéOG9,x-4:TnjuSC,wMwz¼E9NEHIHjmC5, sd55jSrH-9s6Tff2/htg92hrb5w120muj½nTSEG
0A.5c.Nc’: is57jt-FmlIEmxjky8Ouzdnl5BsavCrq7dI⅛9T
jpfcGOnBMC9mwyzfad-x0v:FgPjhyphez/s g½ lrPj1iAGwr-reOkN:kgeRvoFAq5RAr8v:.⅛/w⅛AMSM0OspfeH8¼T’3dtIsH0ltEFyT
/HlAkMqé1Mtpée,Jg8 OPIuE8h1 ’F2-1BTH¼ ro33b3.S5,I’gk’é
c76 jFq,PH2rRTN9s’0⅛N⅛
HO R/:G,6/-3tjiGns3iiOj:ThM En’qC0pBGé,/qHgdoJo,mPTopBG::1fFpooI..é7BeixIjs3MAl⅛1rEaw:
4p2E1FyF7FAxp/:mlcBB
dnnM
J8h ,Faf⅛⅛hET¼4f’61vqybGG9,k89¼RHOh9éS.l7¼i
’xjPCPThx 4⅛Ofnrj28 9.0A87C52Ezq4tzgOA,C/4PCev8¼

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control. It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement curriculum learning to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

model.fit(dataset, epochs=1)